---
<h1 style="text-align: center;">Lista 5 - Exercícios Redes Neurais:</h1>
<h2 style="text-align: center;">Prof. Dr. Jodavid Ferreira</h2>

*Discente:*

    *   Gabriel D'assumpção de Carvalho

*Data:* 05/08/2024

---
---

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Conv2D, Input, Flatten, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt

1. **Crie uma rede neural totalmente conectada com duas camadas ocultas, cada uma com 128 neurônios, utilizando a biblioteca keras. Utilize a função de ativação ReLU e a função de perda categorical_crossentropy para treinar o modelo no conjunto de dados abaixo:**

* Base de dados: https://www.kaggle.com/datasets/abdelazizsami/wine-quality

In [4]:
wine_white = pd.read_csv('neural_network/listas/05/data/winequality-white.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'neural_network/listas/05/data/winequality-white.csv'

In [2]:
# Carregando conjunto de dados
(X_train, y_train), (X_test, y_test)  = mnist.load_data()

In [3]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [4]:
X_train.shape

(60000, 28, 28)

In [5]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [6]:
X_train.shape

(60000, 28, 28, 1)

In [7]:
filtro_h = np.array([[-1, 0, 1],
                     [-1, 0, 1],
                     [-1, 0, 1]], dtype=np.float32)

In [8]:
filtro_h.shape

(3, 3)

In [9]:
'''
# Dimensões:
-> 3,3: O tamanho do kernel (3x3).
-> 1 (terceira dimensão): Escala de cor da imagem que nesse caso é cinza.
-> 1 (quarta dimensão): Quantidade de filtro que esta sendo aplicado.
'''
filtro_h = filtro_h.reshape((3,3,1,1))

In [10]:
filtro_h.shape[:2]

(3, 3)

In [11]:
len(filtro_h)

3

In [12]:
def cnn(X_train, n_f, filtro_h):
    model = models.Sequential()
    model.add(Conv2D(filters=n_f,  # Defina explicitamente o número de filtros, ou ajuste conforme necessário
                     kernel_size=filtro_h.shape[:2],  # Pega o tamanho correto do kernel
                     padding='same',  # Mantém o tamanho da saída igual à entrada
                     use_bias=False,  # Não utiliza viés
                     kernel_initializer=tf.constant_initializer(filtro_h),  # Inicializa com o filtro fornecido
                     trainable=False,  # O filtro não será treinado
                     input_shape=X_train.shape[1:]  # Corrigido o formato da entrada
    ))
    return model

In [13]:
model = cnn(X_train, 1, filtro_h)

/home/gabrieldadcarvalho/anaconda3/envs/am/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1725544867.462503   25190 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 11:01:07.465915: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the req

In [14]:
model.add(MaxPooling2D(pool_size=(2,21)))

In [15]:
model.add(Flatten())

In [16]:
model.add(Dense(15, activation='relu'))

In [17]:
n_categoria = len(np.unique(y_train))

In [18]:
model.add(Dense(n_categoria, activation='softmax'))

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 1)      │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 1, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           225 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 394 (1.54 KB)

 Trainable params: 385 (1.50 KB)

 Non-trainable params: 9 (36.00 B)

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
history = model.fit(X_train, y_train, 
                    batch_size=32, 
                    epochs=20, 
                    validation_split=0.3,
                    )

Epoch 1/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3295 - loss: 1.9650 - val_accuracy: 0.5408 - val_loss: 1.3616
Epoch 2/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5528 - loss: 1.3214 - val_accuracy: 0.5856 - val_loss: 1.2139
Epoch 3/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5980 - loss: 1.2028 - val_accuracy: 0.6016 - val_loss: 1.1600
Epoch 4/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6152 - loss: 1.1499 - val_accuracy: 0.6184 - val_loss: 1.1245
Epoch 5/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6226 - loss: 1.1246 - val_accuracy: 0.6263 - val_loss: 1.0977
Epoch 6/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6342 - loss: 1.0999 - val_accuracy: 0.6298 - val_loss: 1.0825
Epoch 7/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6366 - loss: 1.0782 - val_accuracy: 0.6336 - val_loss: 1.0707
Epoch 8/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6370 - loss: 1.0687 - 

In [1]:
import tensorflow as tf

# Verificar se a GPU está disponível
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Exibir detalhes sobre a GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2024-09-05 11:40:57.102700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 11:40:57.316541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 11:40:57.364903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 11:40:57.820158: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 11:41:01.331933: W tensorflow/compiler/tf2

Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4960001861402292207
xla_global_id: -1
]


I0000 00:00:1725547265.819618    4975 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 11:41:06.044869: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0000 00:00:1725547266.045891    4975 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-0